In [4]:
import json
from sseclient import SSEClient as EventSource
from collections import Counter
import datetime
import signal


url = ' https://stream.wikimedia.org/v2/stream/revision-create'

def signal_handler(signal, frame):
    global interrupted
    interrupted = True

signal.signal(signal.SIGINT, signal_handler)


def fetch_data():
    pageList = {}
    userReport = {}
    time1 = datetime.datetime.now()
    mins = 0
    global interrupted
    interrupted = False
    print("Generating reprot Please wait!..")
    for event in EventSource(url):
        if event.event == 'message':
            try:
                change = json.loads(event.data)            
            except ValueError:
                pass
            else:
                if change["page_title"] not in pageList:
                    pageList[change["page_title"]] = change["meta"]["domain"]
                if change["meta"]["domain"] == "en.wikipedia.org":
                    #print(change["performer"]["user_is_bot"])
                    if change["performer"]["user_is_bot"] == False:
                        count = change["rev_len"]
                        userReport[change["performer"]["user_text"]] = count
                time2 = datetime.datetime.now()
                t = time2- time1               
                if (t.seconds >= 10):
                    mins = mins+1
                    time1 = datetime.datetime.now()
                    print_func(pageList,userReport,mins)
                    if interrupted:
                        print("\nYou have terminated the Program.Thank you!.")
                        break
    

def Convert(report):
    domainlist = list(report.items())
    domainlist.sort(key = lambda x: x[1])
    di = dict(domainlist)
    return di


def print_func(pageList,userReport,mins):
    
    res = Counter(pageList.values())
    pageList = dict(res)
    sortedList=Convert(pageList)
    print("---------------------------------------------------------------------------------------------------------")
    print("\nReport for ",mins," minutes of data....")
    print("#####\t\t\tTotal number of Wikipedia Domains Updated:",len(sortedList))
    for ele in sortedList:
        print("\n",ele,":",sortedList[ele]," pages updated.")
    print("\n#####\t\t\tUsers who made changes to en.wikipedia.org are:\n")
    for ele in userReport:
        print("\n",ele,":",userReport[ele])
    
fetch_data()

Generating reprot Please wait!..
---------------------------------------------------------------------------------------------------------

Report for  1  minutes of data....
#####			Total number of Wikipedia Domains Updated: 26

 el.wiktionary.org : 1  pages updated.

 vi.wikipedia.org : 1  pages updated.

 ja.wikipedia.org : 1  pages updated.

 ban.wikipedia.org : 1  pages updated.

 rw.wikipedia.org : 1  pages updated.

 pl.wikipedia.org : 1  pages updated.

 fr.wiktionary.org : 1  pages updated.

 af.wikipedia.org : 1  pages updated.

 ca.wikipedia.org : 1  pages updated.

 fi.wikipedia.org : 1  pages updated.

 he.wikipedia.org : 1  pages updated.

 ko.wikipedia.org : 1  pages updated.

 ru.wikipedia.org : 2  pages updated.

 avk.wikipedia.org : 2  pages updated.

 en.wiktionary.org : 2  pages updated.

 fr.wikipedia.org : 2  pages updated.

 ar.wikipedia.org : 2  pages updated.

 nl.wikipedia.org : 2  pages updated.

 es.wikipedia.org : 2  pages updated.

 uk.wikipedia.org : 2  p